**Import libraries**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Data loader and Dataset

In [ ]:
def get_train_valid_loader(data_dir, batch_size, augment, random_seed, valid_size = 0.1, shuffle = True):
  """creates and returns training and validation data loaders for the CIFAR-10 dataset.
    input parameters:
      data_dir: The directory where the CIFAR-10 dataset is stored or will be downloaded.
      batch_size: The batch size to use for the data loaders.
      augment: A boolean flag indicating whether data augmentation should be applied to the training dataset.
      random_seed: The random seed used for shuffling the dataset indices.
      valid_size: The proportion of the training dataset to use for validation (default is 0.1, representing 10%).
      shuffle: A boolean flag indicating whether to shuffle the dataset indices before splitting into training and validation sets.

  """
  #define the normalization transformation using the mean and standard deviation values.
  normalize = transforms.Normalize(mean = [0.4914,0.4822,0.4465], std = [0.2023,0.1994,0.2010])

  valid_transform = transforms.Compose([transforms.Resize((227,227)),transforms.ToTensor(),normalize])  #resizes the images to 227x227 pixels,
                                                                                                        #converts them to tensors,
                                                                                                        #and applies the normalization transformation.
  #checks data augmentation
  if augment:
    train_transform = transforms.Compose([transforms.RandomCrop(32, padding=4), #random cropping,
                                          transforms.RandomHorizontalFlip(),    # random horizontal flipping,
                                          transforms.ToTensor(),                #converting images to tensors,
                                          normalize])                           #and applying the normalization transformation
  else:
    train_transform = transforms.Compose([transforms.Resize((227,227)),
                                            transforms.ToTensor(),
                                            normalize])
  train_dataset = datasets.CIFAR10(root = data_dir, train = True, download=True, transform=train_transform)
  valid_dataset = datasets.CIFAR10(root = data_dir, train = True, download=True, transform=valid_transform)

  num_train = len(train_dataset)
  indices = list(range(num_train)) #create a list of indices for shuffling
  split = int(np.floor(valid_size * num_train))#determine the split point for creating the validation dataset

  #checks shuffling
  if shuffle:
    np.random.seed(random_seed)
    np.random.shuffle(indices)

  train_idx, valid_idx = indices[split:], indices[:split]
  #create instances of SubsetRandomSampler using the training and validation indices
  train_sampler = SubsetRandomSampler(train_idx)
  valid_sampler = SubsetRandomSampler(valid_idx)

  #data loaders for the training and validation datasets using the respective samplers
  train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
  valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, sampler=valid_sampler)

  return (train_loader, valid_loader)

def get_test_loader(data_dir, batch_size, shuffle=True):
  normalize = transforms.Normalize(mean = [0.485,0.456,0.406], std = [0.229,0.224,0.225])
  transform = transforms.Compose([transforms.Resize((227,227)),transforms.ToTensor(),normalize])

  dataset = datasets.CIFAR10(root = data_dir, train = False, download=True, transform=transform)

  data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

  return data_loader

train_loader, valid_loader = get_train_valid_loader(data_dir='./data', batch_size=64, augment=False, random_seed=1)
test_loader = get_test_loader(data_dir='./data', batch_size=64)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


Architecture

In [118]:
class GoogLeNet(nn.Module):
    def __init__(self,inChannel,numClass=10):
        super(GoogLeNet, self).__init__()
        self.ConvLayer1= nn.Sequential(nn.Conv2d(3,64,kernel_size=(7,7),stride=(2,2),padding= (3,3)), #takes input with 3 channels and outputs feature maps with 64 channels.
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1),
            nn.Conv2d(64,192,kernel_size=(3,3),stride=(1,1),padding=(1,1)), #takes the 64-channel input and outputs feature maps with 192 channels.
            nn.MaxPool2d(kernel_size=3,stride=2,padding=1))

        #This sequential module consists of two inception blocks and a max pooling layer
        self.ConvLayer2= nn.Sequential(
            InceptionBlock(192, 64, 96, 128, 16, 32, 32),
            InceptionBlock(256, 128, 128, 192, 32, 96, 64),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        #This sequential module consists of multiple inception blocks and a max pooling layer.
        self.ConvLayer3= nn.Sequential(
            InceptionBlock(480, 192, 96, 208, 16, 48, 64),
            InceptionBlock(512, 160, 112, 224, 24, 64, 64),
            InceptionBlock(512, 128, 128, 256, 24, 64, 64),
            InceptionBlock(512, 112, 144, 288, 32, 64, 64),
            InceptionBlock(528, 256, 160, 320, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        #This sequential module consists of multiple inception blocks and an average pooling layer.
        self.ConvLayer4= nn.Sequential(
            InceptionBlock(832,256,160,320,32,128,128),
            InceptionBlock(832, 384, 192, 384, 48, 128, 128),
            nn.AvgPool2d(kernel_size=7,stride= 1),
        )
        #This sequential module consists of a dropout layer followed by a fully connected layer.
        self.fc= nn.Sequential(
            nn.Dropout(0.4),
            nn.Linear(1024,numClass)
        )
    def forward(self,x):
        x= self.ConvLayer1(x)
        #print(x.size())
        x = self.ConvLayer2(x)
        #print(x.size())
        x = self.ConvLayer3(x)
        #print(x.size())
        x = self.ConvLayer4(x)
        #print(x.size())
        x= x.reshape(x.shape[0],-1)
        #print(x.size())
        return self.fc(x)

class InceptionBlock(nn.Module):
    def __init__(self,inChannel,onexone,reducer3x3,threexthree,reducer5x5,fivexfive,pooler):
        super(InceptionBlock,self).__init__()
        self.oneIncetion = ConvBlock(inChannel,onexone,kernel_size=1) #oneIncetion: This variable represents a 1x1 convolutional operation applied to the input tensor.

        self.threeInception = nn.Sequential(
            ConvBlock(inChannel,reducer3x3,kernel_size=1),
            ConvBlock(reducer3x3,threexthree,kernel_size=(3,3),padding=1) #This variable represents a sequence of operations,
                                                                          #including a 1x1 convolutional operation followed by a 3x3 convolutional operation with padding.
        )
        self.fiveInception= nn.Sequential(
            ConvBlock(inChannel,reducer5x5,kernel_size=1),
            ConvBlock(reducer5x5,fivexfive,kernel_size=5,padding=2)#This variable represents a sequence of operations,
                                                                    #including a 1x1 convolutional operation followed by a 5x5 convolutional operation with padding.
        )
        self.poolInception= nn.Sequential(
            nn.MaxPool2d(kernel_size=3,stride=1,padding=1),
            ConvBlock(inChannel,pooler,kernel_size=1) #This variable represents a sequence of operations,
                                                      #including max pooling with a kernel size of 3x3 and a stride of 1, followed by a 1x1 convolutional operation.
        )
    def forward(self,x):
        return torch.cat([self.oneIncetion(x),self.threeInception(x),self.fiveInception(x),self.poolInception(x)],1)


class ConvBlock(nn.Module):
    def __init__(self,inChannel,outChannel,**kwargs):
        super(ConvBlock,self).__init__()
        self.conv= nn.Conv2d(inChannel,outChannel,**kwargs)
        self.BatchNorm= nn.BatchNorm2d(outChannel)
        self.relu = nn.ReLU()
    def forward(self,x):
        return self.relu(self.BatchNorm(self.conv(x)))



In [119]:
num_classes = 10 # the number of classes in CIFAR10
num_epochs = 2 # the number of times the entire training dataset will be passed through the model during training.
batch_size = 16 #the number of samples in each mini-batch used for training.
learning_rate = 0.005 #learning rate used for updating the model's parameters during training.

model = GoogLeNet(num_classes).to(device) #creates an instance of the GoogLeNet model with num_classes as the number of output classes.
                                          #The model is then moved to the device (presumably GPU) for computation.


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)


# Train the model
total_step = len(train_loader) #total number of iterations required to cover the entire training dataset during one epoch.
                              #It uses the length of the train_loader, which is the data loader created for the training dataset.

In [120]:
total_step = len(train_loader)

for epoch in range(num_epochs):  #iterates over the specified number of epochs.
    for i, (images, labels) in enumerate(train_loader):# iterates over the mini-batches of the training dataset. It retrieves images and labels from the train_loader.
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

    # Validation
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in valid_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            del images, labels, outputs

        print('Accuracy of the network on the {} validation images: {} %'.format(5000, 100 * correct / total))

Epoch [1/2], Step [704/704], Loss: 2.2238
Accuracy of the network on the 5000 validation images: 60.84 %
Epoch [2/2], Step [704/704], Loss: 1.0121
Accuracy of the network on the 5000 validation images: 73.66 %


In [121]:
with torch.no_grad(): # disable gradient computation
    correct = 0
    total = 0
    for images, labels in test_loader:
      images = images.to(device)
      labels = labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total+=labels.size(0)
      correct += (predicted == labels).sum().item()
      del images, labels, outputs

    print('Accuracy of the network on {} test images:{} %'.format(10000,100*correct/total))



Accuracy of the network on 10000 test images:73.75 %
